In [ ]:
import os
import numpy as np
import pandas as pd
import aequitas.plot as ap
from aequitas.bias import Bias
from aequitas.group import Group
from postmodeling.evaluation import (
    get_predictions,
    get_test_pred_labels_from_csv
)
from postmodeling.fairness import (
    get_score_attr_df,
    get_group_metrics,
    get_demographics_data,
    get_absolute_metrics,
    enrich_demographics
)
from postmodeling.plotting import (
    plot_disparity,
    save_all_disparity_plots
)
from utils.helpers import (
    get_database_connection,
    load_models
)
import matplotlib
from sklearn.tree import plot_tree

## Fairness audit
This notebook does the fairness audit for a single model (a single split) across sex, race, and county. There are two few observations for hispanic to assess fairness.

In [ ]:
db_conn = get_database_connection()

# This queries the database and takes most of the time
df_dem = get_demographics_data(db_conn, attributes=['sex', 'race', 'ethnicity'])

In [ ]:
model_id = 434
#model_id = 11474
attr_and_ref_groups = {'sex': 'MALE', 'race': 'W', 'hispanic': 'YES', 'county': 'joco'}

In [ ]:
# Get the score and attribute dataframe
df = get_score_attr_df(df_dem, model_id)

# Enrich the table with demographics from other tables (that do not have an event date)
df = enrich_demographics(df, df_dem)

In [ ]:
df

In [ ]:
# Calculate the metrics
xtab, df_metrics = get_group_metrics(df, attr_and_ref_groups)

In [ ]:
df_metrics

## Differences across sex

In [ ]:
# To see how many observations we have per sex and missing
df.sex.value_counts()

In [ ]:
plot_disparity(df_metrics, 'precision', 'sex')

In [ ]:
plot_disparity(df_metrics, 'precision', 'sex')

In [ ]:
plot_disparity(df_metrics, 'tpr', 'sex')

## Differences across race

In [ ]:
# To see how many observations we have per sex and missing
df.race.value_counts()

Since in general we will have only very few observations except for white, black, and missing, we restrict our analysis to those.

In [ ]:
df_metrics_r = df_metrics[df_metrics['attribute_value'].str.contains('B|W|MISSING')]

In [ ]:
plot_disparity(df_metrics_r, 'precision', 'race')

In [ ]:
plot_disparity(df_metrics_r, 'tpr', 'race')

In [ ]:
# Check how many true labels
df_metrics[['attribute_name', 'attribute_value', 'group_label_pos']]

## Differences across counties

In [ ]:
plot_disparity(df_metrics, 'precision', 'county')

In [ ]:
plot_disparity(df_metrics, 'tpr', 'county')

## All metrics

In [ ]:
get_absolute_metrics(xtab)

## Save figures to disk

In [ ]:
# Save disparity plots for all model sets and model ids for one expriment
save_all_disparity_plots(db_conn, experiment_id=25, df_dem=df_dem)